In [1]:
from torch import nn
from tqdm import tqdm_notebook as tqdm
import ipywidgets as widgets
from ipywidgets import interact
import pdb

import time

%run utils.py

%matplotlib inline

In [2]:
w = widgets.IntText()

In [19]:
@interact(x=w)
def f(x):
    WINDOW_SIZE = 100
    
    epoch_progress = len(losses[-1])
    i = max(epoch_progress-WINDOW_SIZE,0)
    if len(losses) > 1:
        diffs = np.array(losses[-1][i:epoch_progress]) - np.array(losses[-2][i:epoch_progress])
        plt.plot(range(i,epoch_progress), diffs, label='This-Last Epoch')
    else:
        plt.plot(range(i,epoch_progress), losses[-1][i:epoch_progress], label='This Epoch')
    plt.xlim((i,epoch_progress))
    plt.legend()
    plt.show()

In [6]:
def df_to_torch(df_clean):
    y = Variable(torch.from_numpy(df_clean['duration'].as_matrix()).float())
    x = Variable(torch.from_numpy(df_clean.drop(['duration',], axis=1).as_matrix()).float())
    return x, y

In [27]:
N, D_in, H, D_out = 1000, 36, 100, 1
model = nn.Sequential(
    nn.Linear(D_in, 1, bias=True),
    #nn.Sigmoid(),
    #nn.Linear(H, D_out),
    #nn.Sigmoid()
)

loss_fn = torch.nn.MSELoss(size_average=False)

In [28]:
losses = []

In [29]:
i = 0
num_entries = 13484192
chunksize = 10000
w.value = 0
learning_rate = 1e-7
display_freq = 10

num_epochs = 50
with tqdm(total=num_epochs*num_entries) as pbar:
    for epoch in range(num_epochs):
        losses.append([])
        df_iter = pd.read_csv('data/yellow_tripdata_2009-01_cleaned.csv', chunksize=chunksize)
        total_loss = 0
        for df in df_iter:
            x, y = df_to_torch(df)
            pbar.update(1)
            # Forward pass: compute predicted y by passing x to the model. Module objects
            # override the _call_ operator so you can call them like functions. When
            # doing so you pass a Variable of input data to the Module and it produces
            # a Variable of output data.
            y_pred = model(x)

            # Compute and print loss. We pass Variables containing the predicted and true
            # values of y, and the loss function returns a Variable containing the loss.
            loss = loss_fn(y_pred, y)
            total_loss += loss.data[0]
            
            # dashboard logic
            losses[-1].append(loss.data[0]/chunksize)
            i += 1
            if (i % display_freq == 0):
                w.value = w.value + 1
            
            # Zero the gradients before running the backward pass.
            model.zero_grad()

            # Backward pass: compute gradient of the loss with respect to all the learnable
            # parameters of the model. Internally, the parameters of each Module are stored
            # in Variables with requires_grad=True, so this call will compute gradients for
            # all learnable parameters in the model.
            loss.backward()

            # Update the weights using gradient descent. Each parameter is a Variable, so
            # we can access its data and gradients like we did before.
            for param in model.parameters():
                param.data -= learning_rate * param.grad.data
            #pdb.set_trace()
        print('epoch', epoch, 'loss', total_loss/num_entries)

KeyboardInterrupt: 

In [ ]:
params = [p for p in model.parameters()]

In [ ]:
x = pd.Series([])

In [ ]:
x = x.append(pd.Series(1), ignore_index=True)

In [ ]:
x